In [ ]:
! pip install transformers torchaudio onnx onnxruntime onnxruntime-gpu

In [ ]:
from transformers import AutoModel
import torch, torchaudio

In [ ]:
# Load the model
model = AutoModel.from_pretrained("ai4bharat/indic-conformer-600m-multilingual", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 403 files:   0%|          | 0/403 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [ ]:
# Load an audio file
wav, sr = torchaudio.load("/content/Hindi_4489_mono.mp3")
wav = torch.mean(wav, dim=0, keepdim=True) # stereo to mono

target_sample_rate = 16000  # Expected sample rate
if sr != target_sample_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sample_rate)
    wav = resampler(wav)

# Perform ASR with CTC decoding
transcription_ctc = model(wav, "hi", "ctc")
print("CTC Transcription:", transcription_ctc)

# Perform ASR with RNNT decoding
transcription_rnnt = model(wav, "hi", "rnnt")
print("RNNT Transcription:", transcription_rnnt)

RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Add node. Name:'/layers.0/self_attn/Add_2' Status Message: /onnxruntime_src/onnxruntime/core/providers/cpu/math/element_wise_ops.h:540 void onnxruntime::BroadcastIterator::Init(ptrdiff_t, ptrdiff_t) axis == 1 || axis == largest was false. Attempting to broadcast an axis by a dimension other than 1. 2521 by 7521


In [ ]:
import os
import torch
import torchaudio
from torchaudio.transforms import Resample

def transcribe_audio_folder(input_folder, model):
    if not os.path.exists(input_folder):
        print(f"Error: Input folder '{input_folder}' does not exist.")
        return

    target_sample_rate = 16000  # Expected sample rate for the ASR model

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".wav"):
            filepath = os.path.join(input_folder, filename)
            print(f"\nProcessing: {filename}")

            try:
                # Load an audio file
                wav, sr = torchaudio.load(filepath)

                if wav.shape[0] > 1: # Check if more than 1 channel -> Mono audio file
                    wav = torch.mean(wav, dim=0, keepdim=True)

                # Resample
                if sr != target_sample_rate:
                    print(f"  Resampling from {sr} Hz to {target_sample_rate} Hz")
                    resampler = Resample(orig_freq=sr, new_freq=target_sample_rate)
                    wav = resampler(wav)

                # Perform ASR with CTC decoding
                transcription_ctc = model(wav, "hi", "ctc")
                print(f"  CTC Transcription: {transcription_ctc}")

                # Perform ASR with RNNT decoding
                transcription_rnnt = model(wav, "mr", "rnnt")
                print(f"  RNNT Transcription: {transcription_rnnt}")

            except Exception as e:
                print(f"  Error processing '{filename}': {e}")

if __name__ == "__main__":
    audio_folder_path = "/content/drive/MyDrive/ASR/MonoAudioFiles"
    # try:
    #     # Example: loading a model if it's from a torchaudio pipeline
    #     # You might need to change this based on your specific model
    #     bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_100H
    #     model = bundle.get_model()
    #     model.eval() # Set the model to evaluation mode
    #     print("ASR model loaded successfully.")
    # except Exception as e:
    #     print(f"Error loading ASR model: {e}. Please ensure your model loading logic is correct.")
    #     exit()

    transcribe_audio_folder(audio_folder_path, model)
    print("\nTranscription process completed for all files!")


Processing: Coll_HDB_MUM_CBD1_4344_07045301328_09-Jun-25-13-22-57.wav
  Resampling from 8000 Hz to 16000 Hz
  CTC Transcription: हेलो हेलो हा जी हा जी बात कर रहे हो ना सरैम गुड आफ्टरनून सर नवीना बात कर रहे हो जैबसे बैंक की तरफ से कॉल पर्सनल लोन का ई एम आई नहीं क्लियर हुआ सर आपका बट कन्फर्म आपका नाइटीन ट्वेंटी होगा क्या बिकॉ आपका मंथ एंड में होता है सर ये नाइनटीन ट्वेंटी को आपका जो इंसेंटिव बता वो आने वाला है कैसा ठीक है सर मैं आपडेट करवा देती हँ बट ट्राई कीजिए उससे पहले अगर आया तो उससे पहले करवा देती ठीक है थैंक यू सर मैम
  RNNT Transcription: हॅलो हॅलो हां जी हां जी बाहेर हां गुड आफ्टरनून सर नवी कर बँके कॉल होते पर्सनल लोन का ई एम आय न क्लिअर हुवा सर आपल्या ट्वेर बट कन्फर्म आपण ट्वेंटी होगा क्या बिकॉज आपण होता सर नाही नाही ट्वेंटी जो इन्सन्टिव बो आला आहे कॅस ठीक आहे सर मॅम अपडेट करत बट ट्राय केस पहिले अगर आयोजन कर ठीक आहे थँक्यू सर मॅम

Processing: Coll_HDB_MUM_CBD1_4200_09870580804_06-Jun-25-10-48-25.wav
  Resampling from 8000 Hz to 16000 Hz
  CTC Transcription: हलो हेलो गुड मॉर्निं